In [7]:
!pip -q install -U wandb datasets torchvision==0.15.2


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# for common
import os
import json
from pathlib import Path

import wandb
from datasets import load_dataset, Image
import boto3, botocore
import sagemaker
from sagemaker import get_execution_role

# for resizing
from PIL import Image
from torchvision import transforms

# for training
from sagemaker import image_uris, model_uris, script_uris

from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.parameter import ContinuousParameter, IntegerParameter
from sagemaker.tuner import HyperparameterTuner

# for inference
import matplotlib.pyplot as plt
import numpy as np
from sagemaker.predictor import Predictor

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
try:
    aws_role = sagemaker.get_execution_role()
except:
    iam = boto3.client("iam")
    # TODO: replace with your role name (i.e. "AmazonSageMaker-ExecutionRole-20211014T154824")
    aws_role = iam.get_role(RoleName="<replace with your RoleName>")["Role"]["Arn"]

boto_session = boto3.Session()
aws_region = boto_session.region_name
sess = sagemaker.Session(boto_session=boto_session)

print(aws_role)
print(aws_region)
print(sess.boto_region_name)

arn:aws:iam::395271362395:role/service-role/AmazonSageMaker-ExecutionRole-20231031T235138
ap-northeast-2
ap-northeast-2


In [4]:
local_training_dataset_folder= Path("text_to_image_training_images")
if not os.path.exists(local_training_dataset_folder):
    os.mkdir(local_training_dataset_folder)

In [5]:
use_local_images = False

if not use_local_images:
    dataset_name = 'haandol/icon'
    dataset = load_dataset(dataset_name, split='train', download_mode='force_redownload')
    
    metadata = []
    for i, datum in enumerate(dataset):
        fn = f'{i}'.zfill(3) + '.jpg'
        datum['image'].convert('RGB').save(local_training_dataset_folder / fn)
        datum['text']
        metadata.append(json.dumps({
            'file_name': fn,
            'text': datum['text']
        }))
    
    with open(local_training_dataset_folder / 'metadata.jsonl', 'w') as fp:
        fp.write('\n'.join(metadata))

Generating train split: 100%|██████████| 263/263 [00:00<00:00, 1794.19 examples/s]


OSError: [Errno 16] Device or resource busy: '.nfse12e1d24c8ce3efb00000004'

In [6]:
validation_prompts = [
    'a photo of red telephone in front of the Eiffel tower',
    'a photo of black telephone handset in front of the Eiffel tower',
    'a white speech bubble in front of the Eiffel tower',
    'a white speech bubble on black background',
    'a woman profile picture in front of the Eiffel tower, looking at viewer',
    'a man profile picture in front of the Eiffel tower, looking at viewer',
    'a white closed envelope on black background',
    'a red closed envelope on white background',
    'a red closed envelope on white background, flat vector',
    'a telephone handset on white background, flat vector',
]
# Instance prompt is fed into the training script via dataset_info.json present in the training folder. Here, we write that file.
with open(os.path.join(local_training_dataset_folder, "dataset_info.json"), "w") as f:
    f.write(json.dumps({
        'validation_prompts': validation_prompts
    }))

In [7]:
mySession = boto3.session.Session()
AwsRegion = mySession.region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

s3 = boto3.client("s3")
s3.download_file(
    f"jumpstart-cache-prod-{AwsRegion}",
    "ai_services_assets/custom_labels/cl_jumpstart_ic_notebook_utils.py",
    "utils.py",
)
from utils import create_bucket_if_not_exists

In [8]:
training_bucket = f"stable-diffusion-text-to-image-{AwsRegion}-{account_id}"

create_bucket_if_not_exists(training_bucket)

train_s3_path = f"s3://{training_bucket}/text-to-image/"

Using an existing bucket stable-diffusion-text-to-image-ap-northeast-2-395271362395


In [12]:
!aws s3 cp --recursive $local_training_dataset_folder $train_s3_path

upload: text_to_image_training_images/002.jpg to s3://stable-diffusion-text-to-image-ap-northeast-2-395271362395/text-to-image/002.jpg
upload: text_to_image_training_images/004.jpg to s3://stable-diffusion-text-to-image-ap-northeast-2-395271362395/text-to-image/004.jpg
upload: text_to_image_training_images/000.jpg to s3://stable-diffusion-text-to-image-ap-northeast-2-395271362395/text-to-image/000.jpg
upload: text_to_image_training_images/001.jpg to s3://stable-diffusion-text-to-image-ap-northeast-2-395271362395/text-to-image/001.jpg
upload: text_to_image_training_images/010.jpg to s3://stable-diffusion-text-to-image-ap-northeast-2-395271362395/text-to-image/010.jpg
upload: text_to_image_training_images/003.jpg to s3://stable-diffusion-text-to-image-ap-northeast-2-395271362395/text-to-image/003.jpg
upload: text_to_image_training_images/007.jpg to s3://stable-diffusion-text-to-image-ap-northeast-2-395271362395/text-to-image/007.jpg
upload: text_to_image_training_images/005.jpg to s3://s

In [32]:
training_instance_type = "ml.g5.12xlarge"

# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = 's3://sagemaker-ap-northeast-2-395271362395/models/stable-diffusion-xl-1.0-base.tar.gz'
train_model_uri

's3://sagemaker-ap-northeast-2-395271362395/models/stable-diffusion-xl-1.0-base.tar.gz'

In [33]:
output_bucket = sess.default_bucket()
output_prefix = "text-to-image-training"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"
s3_output_location

's3://sagemaker-ap-northeast-2-395271362395/text-to-image-training/output'

In [40]:
vae_model = 'madebyollin/sdxl-vae-fp16-fix'
hyperparameters = {
    'learning_rate': 1e-06,
    'max_train_steps': 15000,
    'batch_size': 1,
    'checkpointing_steps': 400,
    'checkpoints_total_limit': 10,
    'gradient_accumulation_steps': 4,
    'resolution': 1024,
    'use_8bit_adam': True,
    'gradient_checkpointing': True,
    'lr_warmup_steps': 100,
    'mixed_precision': 'fp16',
    'pretrained_vae_model_name_or_path': vae_model,
    'lr_scheduler': 'cosine_with_restarts',
    'lr_num_cycles': 3000, # 5 cycles
}
hyperparameters

{'learning_rate': 1e-06,
 'max_train_steps': 15000,
 'batch_size': 1,
 'checkpointing_steps': 400,
 'checkpoints_total_limit': 10,
 'gradient_accumulation_steps': 4,
 'resolution': 512,
 'use_8bit_adam': True,
 'gradient_checkpointing': True,
 'lr_warmup_steps': 100,
 'mixed_precision': 'fp16',
 'pretrained_vae_model_name_or_path': 'madebyollin/sdxl-vae-fp16-fix',
 'lr_scheduler': 'cosine_with_restarts',
 'lr_num_cycles': 3000}

In [41]:
training_job_name = "sd-xl-text-to-image"
training_job_name

'sd-xl-text-to-image'

In [42]:
s3_checkpoint_location = f"s3://{output_bucket}/{training_job_name}/checkpoints"
s3_checkpoint_location

's3://sagemaker-ap-northeast-2-395271362395/sd-xl-text-to-image/checkpoints'

In [43]:
tag = '8ca84f44c2'
train_image_uri = f'395271362395.dkr.ecr.ap-northeast-2.amazonaws.com/text-to-image-training-gpu:{tag}'
train_image_uri

'395271362395.dkr.ecr.ap-northeast-2.amazonaws.com/text-to-image-training-gpu:8ca84f44c2'

In [44]:
use_wandb = True
if use_wandb:
    wandb.login()
    hyperparameters['report_to'] = 'wandb'
    environment = {
        'WANDB_API_KEY': '',  # Update API key
        'WANDB_PROJECT': 'sd-xl-text-to-image',
    }

In [ ]:
# Create SageMaker Estimator instance
sd_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    model_uri=train_model_uri,
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    volume_size=128,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    checkpoint_local_path="/opt/ml/checkpoints",
    checkpoint_s3_uri=s3_checkpoint_location,
    environment=environment if use_wandb else None,
)
sd_estimator.fit({"training": train_s3_path}, logs=True)

INFO:sagemaker:Creating training-job with name: sd-xl-text-to-image-2023-11-01-13-53-56-432


2023-11-01 13:53:56 Starting - Starting the training job...
2023-11-01 13:54:20 Starting - Preparing the instances for training......
2023-11-01 13:55:21 Downloading - Downloading input data.....................
2023-11-01 13:58:37 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
2023-11-01 13:59:34,596 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
ERROR: ld.so: obj